In [ ]:
import osmnx as ox
import folium
import geopandas as gpd
# 峠道
graph = ox.graph_from_point(center_point=(35.334446, 136.990590)
                                , network_type='drive'
                                , dist=1000
                                , simplify=True
                                , custom_filter='["highway"~"tertiary|secondary|primary"]')

# グラフデータをGeoDataFrameに変換
gdf_nodes = ox.graph_to_gdfs(graph, nodes=True, edges=False)
gdf_edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)

def get_nearest_node_by_coordinates(graph, latitude, longitude):
    nearest_node = ox.distance.nearest_nodes(
        graph, longitude, latitude, return_dist=False
    )
    return nearest_node

# 中央付近のノードを取得
center_node = get_nearest_node_by_coordinates(graph, 35.334446, 136.990590)

# 探索を開始するエッジを取得
start_edges = gdf_edges.loc[(gdf_edges.index.get_level_values(0) == center_node)]

# start_edges

line_dicts = []
line_dict = {'total_distance': 0, 'start_node': center_node, 'connected_edges': [center_node] }

max_distance = 1000

# 全エッジ数を出力
print('start node: ', center_node)
print('edge count: ', len(gdf_edges))


# 指定したrefのエッジを先頭に来るように並び替える
def sort_edge_by_ref(edges: gpd.GeoSeries, ref):
  # オブジェクトのエラー対応もしないと・・・・
  edges.loc[(edges == ref)] = 0
  edges.loc[ edges != 0] = 1
  return edges

# 探索処理
def search(line_dicts, edges: gpd.GeoDataFrame, line_dict, previous_node, previous_ref):
  print('node_count: ', len(edges))
  # pandasのdf.sort_valueを使って特定の列に特定の値を含む行を先頭に並び替えたい.custom_sort_keyの引数にはkeyを含むlistと
  edges = edges.sort_values('ref', key=lambda x: sort_edge_by_ref(x.copy(), previous_ref))
  
  for index, edge in edges.iterrows():
    st_node = index[0]
    ed_node = index[1]
    ref = edge.ref
    # 検出済のエッジはスキップ。line_dicts内のconnected_edgesに存在するかどうかで判定
    if(any(line_dict.get('connected_edges') == edge for line_dict in line_dicts)):
      continue
    print('st: ', st_node, 'ed: ', ed_node, 'ref', ref)
    
    # refが同じ場合はラインが1000mを超えるまで探索を続ける。
    # 0は初期値なのでそのまま続ける
    if((previous_ref == 0) | (edge.ref == previous_ref)):
      # ノードを追加
      line_dict['connected_edges'].append(edge)
      # 距離を加算
      line_dict['total_distance'] += edge.length
      print('total_distance: ', line_dict['total_distance'])
      # 1000mを超えていたらラインを確定させる
      if(line_dict['total_distance'] > max_distance):
        line_dicts.append(line_dict)
        line_dict = {'total_distance': 0, 'start_node': st_node, 'connected_edges': [st_node] }
        print(line_dict)
    else:
      line_dict = {'total_distance': 0, 'start_node': st_node, 'connected_edges': [st_node] }
    
    # ここもなんかおかしい?
    # 子ノードが存在する場合は再帰的に探索を続ける
    if(len(gdf_edges.loc[(gdf_edges.index.get_level_values(0) == ed_node)]) > 0):
      r_line_dicts, r_line_dict = search(line_dicts, gdf_edges.loc[(gdf_edges.index.get_level_values(0) == ed_node)], line_dict, st_node, ref)
      line_dicts = r_line_dicts
      line_dict = r_line_dict
  return line_dicts, line_dict
    
r_line_dicts, r_line_dict = search(line_dicts, start_edges, line_dict, 0, 0)